In [1]:
!pip3 install Pillow==4.0.0
!pip3 install torch==0.4.0
!pip3 install torchvision==0.2.1
# !pip install http://download.pytorch.org/whl/cu80/torch-0.3.0.post4-cp36-cp36m-linux_x86_64.whl 

    100% |████████████████████████████████| 484.0MB 29kB/s 
tcmalloc: large alloc 1073750016 bytes == 0x5c486000 @  0x7f8f4d0a91c4 0x46d6a4 0x5fcbcc 0x4c494d 0x54f3c4 0x553aaf 0x54e4c8 0x54f4f6 0x553aaf 0x54efc1 0x54f24d 0x553aaf 0x54efc1 0x54f24d 0x553aaf 0x54efc1 0x54f24d 0x551ee0 0x54e4c8 0x54f4f6 0x553aaf 0x54efc1 0x54f24d 0x551ee0 0x54efc1 0x54f24d 0x551ee0 0x54e4c8 0x54f4f6 0x553aaf 0x54e4c8
    100% |████████████████████████████████| 61kB 5.7MB/s 
    100% |████████████████████████████████| 2.0MB 9.9MB/s 
  Found existing installation: Pillow 4.0.0
    Uninstalling Pillow-4.0.0:
      Successfully uninstalled Pillow-4.0.0


https://pytorch.org/docs/stable/nn.html

In [0]:
import torch 
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import torchvision.transforms as transforms

%matplotlib inline
import matplotlib.pyplot as plt

import numpy as np

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# ユーティリティズ

In [0]:
# class Net(nn.Module):
#     def __init__(self):
#         super(Net, self).__init__()
#         self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
#         self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
#         self.conv2_drop = nn.Dropout2d()
#         self.fc1 = nn.Linear(320, 50)
#         self.fc2 = nn.Linear(50, 10)

#     def forward(self, x):
#         x = F.relu(F.max_pool2d(self.conv1(x), 2))
#         x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
#         x = x.view(-1, 320)
#         x = F.relu(self.fc1(x))
#         x = F.dropout(x, training=self.training)
#         x = self.fc2(x)
#         return F.log_softmax(x, dim=1)
      
# use_cuda = True
# device = torch.device("cuda" if use_cuda else "cpu")
# model.eval()
# model = Net().to(device)
# # output = model(data)

In [3]:
# テンソルを作る
x = torch.tensor(1, requires_grad=True)
w = torch.tensor(2, requires_grad=True)
b = torch.tensor(3, requires_grad=True)

# 計算グラフを作成する
y = w * x + b    # y = 2 * x + 3

# 勾配計算
y.backward()

# 勾配を確認
print(x.grad)    # x.grad = 2 
print(w.grad)    # w.grad = 1 
print(b.grad)    # b.grad = 1 

tensor(2)
tensor(1)
tensor(1)


In [0]:
def clip_grad(v, min, max):
    v_tmp = v.expand_as(v)
    v_tmp.register_hook(lambda g: g.clamp(min, max))
    return v_tmp

In [0]:
clip_grad_value_

In [7]:
grad_clip = 1
clip_grad(x, -grad_clip, grad_clip)

TypeError: ignored

## パラメータの繰り返し可能性の勾配ノルムをクリップ

```
torch.nn.utils.clip_grad_norm_(parameters, max_norm, norm_type=2)
```

正則化だと思う！


パラメータの繰り返し可能性の勾配ノルムをクリップします。 ノルムは、あたかも単一のベクトルに連結されているかのように、すべての勾配にわたって一緒に計算されます。グラデーションはインプレースで変更されます。



返り値：パラメータの総ノルム

In [65]:
x = torch.Tensor([2, 2])
x.requires_grad = True

print(x)
y = torch.sum(x + 1)
print(y)
y.backward() #多次元配列だとエラーになる
print(x.grad)

y.backward()
print(x.grad)

y.backward()
print(x.grad)

tensor([ 2.,  2.])
tensor(6.)
tensor([ 1.,  1.])
tensor([ 2.,  2.])
tensor([ 3.,  3.])


In [86]:
# (10, 3) と (10, 2)のshapeのテンソルを作る
x = torch.randn(10, 3)
y = torch.randn(10, 2)
# x = torch.randn(10, 3, requires_grad = True)
# y = torch.randn(10, 2, requires_grad = True)

# 全結合レイヤー
linear = nn.Linear(3, 2)
# print ('w: ', linear.weight)
# print ('b: ', linear.bias)

# 損失関数と最適化関数
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(linear.parameters(), lr=0.01)

# 順伝播
pred = linear(x)
# lossを計算
loss = criterion(pred, y)
# 逆伝播
loss.backward()
# 勾配の確認
print ('grad dL/dw: ', linear.weight.grad) 
print ('grad dL/db: ', linear.bias.grad)

# １ステップ更新
optimizer.step()
# １ステップ更新したのでlossが変わる
pred = linear(x)
loss = criterion(pred, y)
print ('grad dL/dw: ', linear.weight.grad) 
print ('grad dL/db: ', linear.bias.grad)
print('loss after 1 step optimization: ', loss.item())


# １ステップ更新
optimizer.step()
# １ステップ更新したのでlossが変わる
pred = linear(x)
loss = criterion(pred, y)
print ('grad dL/dw: ', linear.weight.grad) 
print ('grad dL/db: ', linear.bias.grad)
print('loss after 1 step optimization: ', loss.item())

# １ステップ更新
optimizer.step()
# １ステップ更新したのでlossが変わる
pred = linear(x)
loss = criterion(pred, y)
print ('grad dL/dw: ', linear.weight.grad) 
print ('grad dL/db: ', linear.bias.grad)
print('loss after 1 step optimization: ', loss.item())

grad dL/dw:  tensor([[-0.0786, -0.0346,  0.6932],
        [-0.0362, -0.3848, -0.2169]])
grad dL/db:  tensor([ 0.1623, -0.0374])
grad dL/dw:  tensor([[-0.0786, -0.0346,  0.6932],
        [-0.0362, -0.3848, -0.2169]])
grad dL/db:  tensor([ 0.1623, -0.0374])
loss after 1 step optimization:  1.105767011642456
grad dL/dw:  tensor([[-0.0786, -0.0346,  0.6932],
        [-0.0362, -0.3848, -0.2169]])
grad dL/db:  tensor([ 0.1623, -0.0374])
loss after 1 step optimization:  1.098765254020691
grad dL/dw:  tensor([[-0.0786, -0.0346,  0.6932],
        [-0.0362, -0.3848, -0.2169]])
grad dL/db:  tensor([ 0.1623, -0.0374])
loss after 1 step optimization:  1.0918428897857666


In [79]:
# (10, 3) と (10, 2)のshapeのテンソルを作る
x = torch.randn(10, 3)
y = torch.randn(10, 2)
# x = torch.randn(10, 3, requires_grad = True)
# y = torch.randn(10, 2, requires_grad = True)
torch.nn.utils.clip_grad_norm_(x, 0.1)
torch.nn.utils.clip_grad_norm_(y, 0.1)


# 全結合レイヤー
linear = nn.Linear(3, 2)
# print ('w: ', linear.weight)
# print ('b: ', linear.bias)

# 損失関数と最適化関数
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(linear.parameters(), lr=0.01)

# 順伝播
pred = linear(x)
# lossを計算
loss = criterion(pred, y)
# 逆伝播
loss.backward()
# 勾配の確認
print ('grad dL/dw: ', linear.weight.grad) 
print ('grad dL/db: ', linear.bias.grad)

# １ステップ更新
optimizer.step()
# １ステップ更新したのでlossが変わる
pred = linear(x)
loss = criterion(pred, y)
print('loss after 1 step optimization: ', loss.item())


# １ステップ更新
optimizer.step()
# １ステップ更新したのでlossが変わる
pred = linear(x)
loss = criterion(pred, y)
print('loss after 1 step optimization: ', loss.item())

# １ステップ更新
optimizer.step()
# １ステップ更新したのでlossが変わる
pred = linear(x)
loss = criterion(pred, y)
print('loss after 1 step optimization: ', loss.item())






grad dL/dw:  tensor([[ 0.0704, -0.6308, -0.0338],
        [ 0.7090, -0.5395, -0.5156]])
grad dL/db:  tensor([ 0.6013,  0.9322])
loss after 1 step optimization:  1.6067063808441162
loss after 1 step optimization:  1.5803437232971191
loss after 1 step optimization:  1.554366111755371


In [0]:
# 
 


In [25]:
import torch
from torch.autograd import Variable
import numpy as np

x = torch.arange(1,4, requires_grad=True)
y = sum(x**2)
z = sum(x*3)

y.backward()
print(x.grad)

 # how to set x.grad = 0??
z.backward()
print(x.grad)

tensor([ 2.,  4.,  6.])
tensor([ 5.,  7.,  9.])


## 可変長のパッド付きシーケンスを含むTensorをパック

```
torch.nn.utils.rnn.pack_padded_sequence(input, lengths, batch_first=False)
```

## 可変長配列のパックされたバッチを埋め込み

```
torch.nn.utils.rnn.pad_packed_sequence(sequence, batch_first=False, padding_value=0.0, total_length=None)
```

# torch.nn.functional

## Convolution functions

##Pooling functions

### 1次元アベレージプーリング


```
torch.nn.functional.avg_pool1d(input, kernel_size, stride=None, padding=0, ceil_mode=False, count_include_pad=True) → Tensor
```


kernel_size：カーネルサイズ    
stride：ストライド。デフォルトはkernel_size    
padding：両端にゼロパディングを追加    
dilation：ウィンドウ内の要素のストライドを制御するパラメータ    
return_indices：Trueの場合、出力とともに最大インデックスが返されます。後でアンプールするときに便利    
ceil_mode：Trueの場合、floorの代わりにceilを使用して出力を計算します。floorは切り捨て。ceilは切り捨て。


In [8]:
# pool of square window of size=3, stride=2
input = torch.tensor([[[1,2,3,4,5,6,7]]], dtype=torch.float)
F.avg_pool1d(input, kernel_size=3, stride=2)

tensor([[[ 2.,  4.,  6.]]])